In [2]:
import pandas as pd
import matplotlib.pyplot as plt
from sqlalchemy import create_engine
from config import DB_USERNAME, DB_PASSWORD, DB_HOST, DB_PORT, DB_NAME

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# import warnings
# warnings.filterwarnings("ignore")

In [3]:
# Create a connection string with the postgreSQL info
database_access = f"postgresql://{DB_USERNAME}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"


In [4]:
# Establish a connection to the PostgreSQL database 
engine = create_engine(database_access)


In [5]:
# Write a SQL query to select data from the existing view in PostgreSQL 
query = 'SELECT * FROM "Wines"'


In [6]:
# Use Pandas to read the data from the SQL query and convert into a DataFrame 
wines_df = pd.read_sql_query(query, engine)


In [7]:
# Display the Dataframe
wines_df.head()

,id,type,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality,quality_categorisation
0,4799,White,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,6,1
1,1159,Red,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,5,0
2,315,Red,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,6,1
3,31,Red,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,6,1
4,3010,White,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,5,0


## LEWIS - Machine Learning Code 

In [8]:
# Clean the dataframe, removing columns that will not be used in the ML model
cleaned_wines_df = wines_df.drop(columns=['id','type', 'quality'])

# Display the clean dataframe
cleaned_wines_df.head()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,ph,sulphates,alcohol_wine,quality_categorisation
0,6.2,0.260,0.20,8.00,0.047,35.0,111.0,0.99445,3.11,0.42,10.4,1
1,10.2,0.410,0.43,2.20,0.110,11.0,37.0,0.99728,3.16,0.67,10.8,0
2,7.1,0.350,0.29,2.50,0.096,20.0,53.0,0.99620,3.42,0.65,11.0,1
3,6.9,0.685,0.00,2.50,0.105,22.0,37.0,0.99660,3.46,0.57,10.6,1
4,6.7,0.250,0.31,1.35,0.061,30.5,218.0,0.99388,3.16,0.53,9.5,0


In [9]:
# Split our preprocessed data into our features and target arrays
X = cleaned_wines_df.drop("quality_categorisation", axis=1)
y = cleaned_wines_df["quality_categorisation"]


In [10]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Fit the StandardScaler
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)



In [11]:
#Define deep neural network model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [12]:
#Train the model
model.fit(X_train, y_train, epochs=100)


Epoch 1/100
143/143 [==============================] - 3s 3ms/step - loss: 1.6756 - accuracy: 0.5135
Epoch 2/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6827 - accuracy: 0.6200
Epoch 3/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6281 - accuracy: 0.6413
Epoch 4/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6092 - accuracy: 0.6468
Epoch 5/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6020 - accuracy: 0.6556
Epoch 6/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6058 - accuracy: 0.6585
Epoch 7/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6088 - accuracy: 0.6554
Epoch 8/100
143/143 [==============================] - 0s 3ms/step - loss: 0.5895 - accuracy: 0.6679
Epoch 9/100
143/143 [==============================] - 0s 3ms/step - loss: 0.5962 - accuracy: 0.6657
Epoch 10/100
143/143 [==============================] - 0s 3ms/step - loss: 0.5866 - accura

In [13]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 3ms/step - loss: 0.5205 - accuracy: 0.7333
Loss: 0.5205374956130981, Accuracy: 0.7333333492279053


In [14]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
# model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [15]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 2s 3ms/step - loss: 5.8219 - accuracy: 0.6382
Epoch 2/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6955 - accuracy: 0.6112
Epoch 3/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6613 - accuracy: 0.6230
Epoch 4/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6457 - accuracy: 0.6305
Epoch 5/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6349 - accuracy: 0.6409
Epoch 6/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6287 - accuracy: 0.6413
Epoch 7/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6238 - accuracy: 0.6431
Epoch 8/100
143/143 [==============================] - 0s 3ms/step - loss: 0.6182 - accuracy: 0.6499
Epoch 9/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6102 - accuracy: 0.6477
Epoch 10/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6045 - accura

In [16]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 0s 4ms/step - loss: 0.5129 - accuracy: 0.7405
Loss: 0.5129055380821228, Accuracy: 0.7405128479003906


In [17]:
#Define deep neural network model increased unit amounts, changed to tanh from relu
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=300, input_dim=X_train_scaled.shape[1], activation="tanh"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=200, activation="tanh"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [18]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 2s 6ms/step - loss: 0.6402 - accuracy: 0.6380
Epoch 2/100
143/143 [==============================] - 1s 4ms/step - loss: 0.5988 - accuracy: 0.6578
Epoch 3/100
143/143 [==============================] - 1s 4ms/step - loss: 0.5965 - accuracy: 0.6675
Epoch 4/100
143/143 [==============================] - 1s 4ms/step - loss: 0.5675 - accuracy: 0.6921
Epoch 5/100
143/143 [==============================] - 1s 6ms/step - loss: 0.5551 - accuracy: 0.6998
Epoch 6/100
143/143 [==============================] - 1s 5ms/step - loss: 0.5416 - accuracy: 0.7082
Epoch 7/100
143/143 [==============================] - 1s 5ms/step - loss: 0.5502 - accuracy: 0.7007
Epoch 8/100
143/143 [==============================] - 1s 6ms/step - loss: 0.5383 - accuracy: 0.7057
Epoch 9/100
143/143 [==============================] - 1s 6ms/step - loss: 0.5309 - accuracy: 0.7167
Epoch 10/100
143/143 [==============================] - 1s 5ms/step - loss: 0.5373 - accura

In [19]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 0s 3ms/step - loss: 0.5180 - accuracy: 0.7364
Loss: 0.5179942846298218, Accuracy: 0.7364102602005005


In [20]:
#Define deep neural network model increased unit amounts, changed to tanh from relu
model = tf.keras.models.Sequential()

# First hidden layer with L2 regularization
model.add(Dense(units=300, input_dim=X_train_scaled.shape[1], activation="tanh", kernel_regularizer=l2(0.001)))

# Second hidden layer with L2 regularization
model.add(Dense(units=200, activation="tanh", kernel_regularizer=l2(0.001)))

# Output layer
model.add(Dense(units=1, activation="sigmoid", kernel_regularizer=l2(0.001)))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [21]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 2s 4ms/step - loss: 0.8941 - accuracy: 0.6252
Epoch 2/100
143/143 [==============================] - 1s 5ms/step - loss: 0.8124 - accuracy: 0.6470
Epoch 3/100
143/143 [==============================] - 1s 5ms/step - loss: 0.7482 - accuracy: 0.6659
Epoch 4/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7193 - accuracy: 0.6666
Epoch 5/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6907 - accuracy: 0.6679
Epoch 6/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6546 - accuracy: 0.7016
Epoch 7/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6398 - accuracy: 0.7040
Epoch 8/100
143/143 [==============================] - 1s 4ms/step - loss: 0.6463 - accuracy: 0.6939
Epoch 9/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6117 - accuracy: 0.7071
Epoch 10/100
143/143 [==============================] - 1s 5ms/step - loss: 0.6297 - accura

In [22]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")


61/61 [==============================] - 1s 4ms/step - loss: 0.5489 - accuracy: 0.7313
Loss: 0.5489052534103394, Accuracy: 0.7312820553779602


In [23]:
# Define deep neural network model
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=120, activation="relu", input_dim=X_train_scaled.shape[1], kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=80, activation="relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Third hidden layer
model.add(tf.keras.layers.Dense(units=40, activation="relu", kernel_regularizer=l2(0.001)))
model.add(tf.keras.layers.Dropout(0.3))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [24]:
#Train the model
model.fit(X_train, y_train, epochs=100)

Epoch 1/100
143/143 [==============================] - 3s 5ms/step - loss: 1.3115 - accuracy: 0.5542
Epoch 2/100
143/143 [==============================] - 1s 6ms/step - loss: 0.8072 - accuracy: 0.6039
Epoch 3/100
143/143 [==============================] - 1s 5ms/step - loss: 0.7779 - accuracy: 0.6217
Epoch 4/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7623 - accuracy: 0.6327
Epoch 5/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7554 - accuracy: 0.6340
Epoch 6/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7437 - accuracy: 0.6354
Epoch 7/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7391 - accuracy: 0.6362
Epoch 8/100
143/143 [==============================] - 1s 4ms/step - loss: 0.7332 - accuracy: 0.6360
Epoch 9/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7309 - accuracy: 0.6371
Epoch 10/100
143/143 [==============================] - 0s 3ms/step - loss: 0.7239 - accura

In [25]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 1s 3ms/step - loss: 0.5601 - accuracy: 0.7015
Loss: 0.5600669980049133, Accuracy: 0.7015384435653687


In [33]:
#Define deep neural network model removed third layer
model = tf.keras.models.Sequential()

# First hidden layer
model.add(tf.keras.layers.Dense(units=12, input_dim=X_train_scaled.shape[1], activation="relu"))

# Second hidden layer
model.add(tf.keras.layers.Dense(units=8, activation="relu"))

# Third hidden layer
# model.add(tf.keras.layers.Dense(units=4, activation="relu"))

# Output layer
model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [34]:
#Train the model
model.fit(X_train, y_train, epochs=100, batch_size=10)

Epoch 1/100
455/455 [==============================] - 3s 3ms/step - loss: 0.9013 - accuracy: 0.6325
Epoch 2/100
455/455 [==============================] - 1s 3ms/step - loss: 0.6007 - accuracy: 0.6635
Epoch 3/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5910 - accuracy: 0.6684
Epoch 4/100
455/455 [==============================] - 2s 4ms/step - loss: 0.5776 - accuracy: 0.6809
Epoch 5/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5699 - accuracy: 0.6899
Epoch 6/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5639 - accuracy: 0.6877
Epoch 7/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5588 - accuracy: 0.7044
Epoch 8/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5483 - accuracy: 0.7079
Epoch 9/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5567 - accuracy: 0.7053
Epoch 10/100
455/455 [==============================] - 1s 3ms/step - loss: 0.5437 - accura

In [35]:
model_loss, model_accuracy = model.evaluate(X_test, y_test)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

61/61 [==============================] - 0s 3ms/step - loss: 0.5123 - accuracy: 0.7503
Loss: 0.512319803237915, Accuracy: 0.7502564191818237


## KAI - Correlation Code
##### (please use the [wines_df] DataFrame as it contains the 'quality' column that you will need to calculate the correlations)
